In [1]:
import numpy as np
from pathlib import Path
from shared.utilities import pad_to_max_sample_length
from shared.data import add_stage_dimension

2023-08-21 13:25:17.960732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 13:25:18.478517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Split data into segments

In [2]:
data_path = Path("data/sat1/stage_data.nc")
output_path = Path("data/sat1/split_stage_data.nc")
# Takes ~9 minutes
output_data = add_stage_dimension(data_path)

In [ ]:
# Save segmented data
output_data.to_netcdf(output_path)

### Deprecated

In [ ]:
# %%time
# 2488 labels/segments
stage_data = xr.load_dataset(data_path)
dimensions = stage_data.labels.shape
segments = []
labels = []
for participant in range(dimensions[0]):
    for epoch in range(dimensions[1]):
        trial = stage_data.isel(participant=participant, epochs=epoch)
        # If trial contains anything other than empty strings, at least one processing stage has been identified and labelled
        if np.any(trial.labels != ""):
            np_labels = trial.labels.to_numpy()
            # Find locations where string changes in a sequence of strings to find changes in stages
            changes = np.where(np_labels[:-1] != np_labels[1:])[0] + 1
            # Get EEG data for participant/epoch

            # Create list of EEG data splits
            splits = np.split(trial.data, changes, 1)
            label_splits = np.split(trial.labels, changes, 0)
            for split, label_split in zip(splits, label_splits):
                if np.any(label_split != ""):
                    # Sanity check
                    if np.any(np.isnan(split)):
                        print(f"NAN FOUND participant {participant}, epoch {epoch}")
                        print(f"Label: {label_split[0].item()}")
                        print(split.to_numpy()[0, :])
                        raise ValueError(
                            "Split found that is fully labeled but still contains NaNs in data"
                        )
                    # Add split and label to lists
                    segments.append(split.data)
                    labels.append(label_split[0].item())
        else:
            print(f"Trial unusable: participant {participant}, epoch {epoch}")

# Pad to max_sample_length, can differ per data set or subset
max_sample_length = max(segment.shape[1] for segment in segments)
segments = np.stack(
    [pad_to_max_sample_length(segment, max_sample_length) for segment in segments]
)